### How to chat with GPT using OpenAI API

In [1]:
import os
import openai
from retrying import retry
from IPython.display import Markdown,display
print ("openai version:[%s]"%(openai.__version__))

openai version:[0.28.0]


### Locate where your key is

In [2]:
key_path = '../key/rilab_key.txt'
print ('key_path:[%s]'%(key_path))

key_path:[../key/rilab_key.txt]


### Use key

In [3]:
with open(key_path, 'r') as f: OPENAI_API_KEY = f.read()
openai.api_key = OPENAI_API_KEY

### Query function

In [4]:
@retry(stop_max_attempt_number=5,
       wait_exponential_multiplier=1000,
       wait_exponential_max=10000)
def query_gpt(messages:list,gpt_model='gpt-4'):
    """
        gpt_model: 'gpt-3.5-turbo' / 'gpt-4'
    """
    # Call the OpenAI API
    response     = openai.ChatCompletion.create(
        model    = gpt_model, 
        messages = messages
    )
    # Extract the response content and status code
    content     = response["choices"][0]["message"]["content"]
    status_code = response["choices"][0]["finish_reason"]
    return content,status_code,response
print ("Ready.")

Ready.


`messages`, an input to GPT, is basically a list where each item is a dictionary consists of `role` and `content`. A `role` can either be
* `system`: which defines the identity of the agent
* `user`: which states the input of a user
* `assistant`: which stores messages previously generated by the agents
More information can be found in [here](https://platform.openai.com/docs/guides/gpt/chat-completions-api).

In [5]:
role_msg = """너는 한국어로 대화에 능통한 언어모델이야"""
question = "LLM(Large Language Model)은 어떻게 작동하는지 설명해줘"
messages = [{"role": "system", "content": f'{role_msg}'},
            {"role": 'user', "content": f'{question}'}]
print (messages)

[{'role': 'system', 'content': '너는 한국어로 대화에 능통한 언어모델이야'}, {'role': 'user', 'content': 'LLM(Large Language Model)은 어떻게 작동하는지 설명해줘'}]


### Now let's use `GPT`

In [6]:
content,status_code,response = query_gpt(messages=messages,gpt_model='gpt-3.5-turbo')

In [7]:
print (content)

LLM은 큰 규모의 언어 모델로, 텍스트 관련 작업을 수행하기 위해 사용됩니다. LLM은 기계 학습 기술을 사용하여 훈련되며, 많은 양의 텍스트 데이터를 이용해 언어의 통계적 패턴과 구조를 학습합니다.

LLM은 주어진 입력에 대한 다음 단어나 문장을 예측하는 데 사용됩니다. 예를 들어, "나는 오늘"이라는 입력이 주어진다면, LLM은 다음에 올 수 있는 가능한 단어를 예측하여 "나는 오늘 먹었다" 또는 "나는 오늘 공부했다"와 같은 결과를 출력할 수 있습니다.

LLM은 작동하기 위해 큰 양의 데이터를 사용합니다. 대규모의 코퍼스(텍스트 데이터 모음)를 사용하여 많은 문장과 문서를 학습하고, 문법, 단어 간의 관계, 의미, 문맥 등 다양한 언어적 특징을 학습합니다.

LLM은 훈련된 이후에는 실제 응용 프로그램에서 다양한 작업에 활용될 수 있습니다. 예를 들어, 기계 번역, 자동 요약, 질의응답 시스템, 텍스트 생성 등에 활용될 수 있습니다. LLM은 입력 텍스트와 관련된 문제에 대한 올바른 예측을 생성하여 도움을 줄 수 있습니다.


In [8]:
print (status_code)

stop


In [9]:
print (response)

{
  "id": "chatcmpl-8FRQqdZgJ0nN2kSmQKIeiekX9Gx8c",
  "object": "chat.completion",
  "created": 1698691060,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "LLM\uc740 \ud070 \uaddc\ubaa8\uc758 \uc5b8\uc5b4 \ubaa8\ub378\ub85c, \ud14d\uc2a4\ud2b8 \uad00\ub828 \uc791\uc5c5\uc744 \uc218\ud589\ud558\uae30 \uc704\ud574 \uc0ac\uc6a9\ub429\ub2c8\ub2e4. LLM\uc740 \uae30\uacc4 \ud559\uc2b5 \uae30\uc220\uc744 \uc0ac\uc6a9\ud558\uc5ec \ud6c8\ub828\ub418\uba70, \ub9ce\uc740 \uc591\uc758 \ud14d\uc2a4\ud2b8 \ub370\uc774\ud130\ub97c \uc774\uc6a9\ud574 \uc5b8\uc5b4\uc758 \ud1b5\uacc4\uc801 \ud328\ud134\uacfc \uad6c\uc870\ub97c \ud559\uc2b5\ud569\ub2c8\ub2e4.\n\nLLM\uc740 \uc8fc\uc5b4\uc9c4 \uc785\ub825\uc5d0 \ub300\ud55c \ub2e4\uc74c \ub2e8\uc5b4\ub098 \ubb38\uc7a5\uc744 \uc608\uce21\ud558\ub294 \ub370 \uc0ac\uc6a9\ub429\ub2c8\ub2e4. \uc608\ub97c \ub4e4\uc5b4, \"\ub098\ub294 \uc624\ub298\"\uc774\ub77c\ub294 \uc78

### Helper Class for implementing efficient chat with GPT

In [10]:
def printmd(string):
    display(Markdown(string))
class GPTchatClass():
    def __init__(self,
                 gpt_model = 'gpt-4',
                 role_msg  = 'Your are a helpful assistant.',
                 VERBOSE   = True
                ):
        self.gpt_model     = gpt_model
        self.messages      = [{'role':'system','content':f'{role_msg}'}]
        self.init_messages = [{'role':'system','content':f'{role_msg}'}]
        self.VERBOSE       = VERBOSE
        self.response      = None
        if self.VERBOSE:
            print ("Chat agent using [%s] initialized with the follow role:[%s]"%
                   (self.gpt_model,role_msg))
    
    def _add_message(self,role='assistant',content=''):
        """
            role: 'assistant' / 'user'
        """
        self.messages.append({'role':role, 'content':content})
        
    def _get_response_content(self):
        if self.response:
            return self.response['choices'][0]['message']['content']
        else:
            return None
        
    def _get_response_status(self):
        if self.response:
            return self.response['choices'][0]['message']['finish_reason']
        else:
            return None
    
    def chat(self,user_msg='hi',
             PRINT_USER_MSG=True,PRINT_GPT_OUTPUT=True,
             RESET_CHAT=False,RETURN_RESPONSE=True):
        self._add_message(role='user',content=user_msg)
        self.response = openai.ChatCompletion.create(
            model    = self.gpt_model,
            messages = self.messages
        )
        # Backup response for continous chatting
        self._add_message(role='assistant',content=self._get_response_content())
        if PRINT_USER_MSG:
            print("[USER_MSG]")
            printmd(user_msg)
        if PRINT_GPT_OUTPUT:
            print("[GPT_OUTPUT]")
            printmd(self._get_response_content())
        # Reset
        if RESET_CHAT:
            self.messages = self.init_messages
        # Return
        if RETURN_RESPONSE:
            return self._get_response_content()
print ("Ready.")              

Ready.


### Now let's chat

In [11]:
GPT = GPTchatClass(gpt_model='gpt-3.5-turbo',role_msg  = 'Your are a helpful assistant.')
PRINT_USER_MSG   = True
PRINT_GPT_OUTPUT = True
RESET_CHAT       = False
RETURN_RESPONSE  = False
GPT.chat(user_msg='Who is the current president of Korea?',
         PRINT_USER_MSG=PRINT_USER_MSG,PRINT_GPT_OUTPUT=PRINT_GPT_OUTPUT,
         RESET_CHAT=RESET_CHAT,RETURN_RESPONSE=RETURN_RESPONSE)
GPT.chat(user_msg='Are you sure? I think you are outdated.',
         PRINT_USER_MSG=PRINT_USER_MSG,PRINT_GPT_OUTPUT=PRINT_GPT_OUTPUT,
         RESET_CHAT=RESET_CHAT,RETURN_RESPONSE=RETURN_RESPONSE)
GPT.chat(user_msg='Where can I get the latest information?',
         PRINT_USER_MSG=PRINT_USER_MSG,PRINT_GPT_OUTPUT=PRINT_GPT_OUTPUT,
         RESET_CHAT=RESET_CHAT,RETURN_RESPONSE=RETURN_RESPONSE)

Chat agent using [gpt-3.5-turbo] initialized with the follow role:[Your are a helpful assistant.]
[USER_MSG]


Who is the current president of Korea?

[GPT_OUTPUT]


As of my last update in November 2021, the current president of South Korea is Moon Jae-in. However, please note that political positions can change, so it's always a good idea to verify with the latest information from reliable sources.

[USER_MSG]


...

I apologize if my previous response was outdated. As an AI, I do not have real-time information updates. As of November 2021, Moon Jae-in was serving as the president of South Korea, but please verify with the latest sources to ensure accuracy.

[USER_MSG]


Where can I get the latest information?

[GPT_OUTPUT]


For the most up-to-date and accurate information on the current president of South Korea, I recommend checking reliable news sources such as reputable news websites, government websites, or official government social media accounts. These sources usually provide the most recent and accurate information on political leadership and updates.

### Chat with resetting everytime

In [12]:
GPT = GPTchatClass(gpt_model='gpt-3.5-turbo',role_msg  = 'Your are a helpful assistant.')
PRINT_USER_MSG   = True
PRINT_GPT_OUTPUT = True
RESET_CHAT       = True
RETURN_RESPONSE  = False
GPT.chat(user_msg='Who is the current president of Korea?',
         PRINT_USER_MSG=PRINT_USER_MSG,PRINT_GPT_OUTPUT=PRINT_GPT_OUTPUT,
         RESET_CHAT=RESET_CHAT,RETURN_RESPONSE=RETURN_RESPONSE)
GPT.chat(user_msg='Are you sure? I think you are outdated.',
         PRINT_USER_MSG=PRINT_USER_MSG,PRINT_GPT_OUTPUT=PRINT_GPT_OUTPUT,
         RESET_CHAT=RESET_CHAT,RETURN_RESPONSE=RETURN_RESPONSE)
GPT.chat(user_msg='Where can I get the latest information?',
         PRINT_USER_MSG=PRINT_USER_MSG,PRINT_GPT_OUTPUT=PRINT_GPT_OUTPUT,
         RESET_CHAT=RESET_CHAT,RETURN_RESPONSE=RETURN_RESPONSE)

Chat agent using [gpt-3.5-turbo] initialized with the follow role:[Your are a helpful assistant.]
[USER_MSG]


Who is the current president of Korea?

[GPT_OUTPUT]


As of my knowledge, the current President of South Korea is Moon Jae-in.

[USER_MSG]


...

I'm here to assist you to the best of my ability. While I may not have the latest information or be able to perform certain tasks, I'll do my best to help you with any questions or tasks you have. Let me know how I can assist you!

[USER_MSG]


Where can I get the latest information?

[GPT_OUTPUT]


To get the latest information, there are several reliable sources you can turn to:

1. News websites: Popular news websites like BBC News, CNN, Reuters, or The New York Times provide up-to-date news on various topics.

2. Social media platforms: Follow reputable news outlets or journalists on platforms like Twitter, where they often share the latest updates and breaking news.

3. Official government websites: Government websites provide reliable information on topics such as public health, policies, and current events. Look for websites specific to your country or region.

4. Trusted experts and organizations: Follow experts or organizations related to the topic you're interested in. For example, if you're looking for health-related information, following the World Health Organization (WHO) or the Centers for Disease Control and Prevention (CDC) can provide reliable updates.

Remember, it's always important to verify the credibility of your sources and cross-reference information from multiple reliable sources to ensure accuracy.